In [1]:
import random
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.mixture import GaussianMixture
from sklearn.metrics import precision_score,accuracy_score,roc_auc_score
from sklearn.metrics import balanced_accuracy_score
from tqdm import tqdm
import pickle
def makefile(what,filename):
    with open(filename,"wb") as f3:
        pickle.dump(what,f3)

def readfile(filename):
    with open(filename,"rb") as f4:
        ans=pickle.load(f4)
    return ans
import random
from sklearn.mixture import GaussianMixture
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.svm import OneClassSVM
import warnings
warnings.filterwarnings("ignore")
from sklearn.cluster import KMeans

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [208]:
#checking the model size by saving the model
positive_models=readfile("all_positive_models.pkl")
negative_models=readfile("all_negative_models.pkl")
e=0
for word, pos_models, neg_models in zip(fetch_dataset, positive_models, negative_models):
    print(word)
    makefile(pos_models,"models/"+word+"_pos.pkl")
    makefile(neg_models,"models/"+word+"_neg.pkl")

ecoli
optical_digits
satimage
pen_digits
abalone
sick_euthyroid
spectrometer
car_eval_34
isolet
us_crime
yeast_ml8
scene
libras_move
thyroid_sick
coil_2000
arrhythmia
solar_flare_m0
oil
car_eval_4
wine_quality
letter_img
yeast_me2
webpage
ozone_level
mammography
protein_homo
abalone_19


In [128]:
#applying the model.
#positive means minority, negative is majority
%%time
table_pos=[]
table_neg=[]
positive_models=readfile("all_positive_models.pkl")
negative_models=readfile("all_negative_models.pkl")
e=0
for word, pos_models, neg_models in zip(fetch_dataset, positive_models, negative_models):
    X=fetch_dataset[word]["data"]
    Y=fetch_dataset[word]["target"]
    print(word,":",len(X[Y==1]),":",len(X[Y==-1]),":",len(X[0]))
    X=(X-X.min(axis=0))/(X.max(axis=0)-X.min(axis=0))
    X[pd.isnull(X)]=0
    random.seed(0)
    d=len(X[0])
    split=0.4
    random_value=1
    #This line should ensure the same split
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=split, random_state=e,stratify=Y)
    positive=X_train[y_train==1]
    negative=X_train[y_train==-1]
    score_mean=[]
    score_std=[]
    for n in range(115):
        score=pos_models[n].score_samples(positive)
        score_mean.append(score.mean())
        score_std.append(score.std())
         
    AUC_table=np.zeros(116)
    score_test=np.zeros(len(X_test))
    for n in range(115):
        score=pos_models[n].score_samples(X_test)
        #print(roc_auc_score(y_test,score))
        AUC_table[n]=roc_auc_score(y_test,score)
        if score_std[n]==0:
            pass
        else:
            score_test+=(score-score_mean[n])/(score_std[n])
    
    AUC_table[115]=roc_auc_score(y_test,score_test)
    table_pos.append(AUC_table)
    
    print("negative")
    score_mean=[]
    score_std=[]
    for n in range(115):
        score=neg_models[n].score_samples(negative)
        score_mean.append(score.mean())
        score_std.append(score.std())
         
    AUC_table=np.zeros(116)
    score_test=np.zeros(len(X_test))
    for n in range(115):
        score=neg_models[n].score_samples(X_test)
        #print(roc_auc_score(y_test,-score))
        AUC_table[n]=roc_auc_score(y_test,-score)
        if score_std[n]==0:
            pass
        else:
            score_test+=(score-score_mean[n])/(score_std[n])
    AUC_table[115]=roc_auc_score(y_test,-score_test)
    table_neg.append(AUC_table)


ecoli : 35 : 301 : 7
negative
optical_digits : 554 : 5066 : 64
negative
satimage : 626 : 5809 : 36
negative
pen_digits : 1055 : 9937 : 16
negative
abalone : 391 : 3786 : 10
negative
sick_euthyroid : 293 : 2870 : 42
negative
spectrometer : 45 : 486 : 93
negative
car_eval_34 : 134 : 1594 : 21
negative
isolet : 600 : 7197 : 617
negative
us_crime : 150 : 1844 : 100
negative
yeast_ml8 : 178 : 2239 : 103
negative
scene : 177 : 2230 : 294
negative
libras_move : 24 : 336 : 90
negative
thyroid_sick : 231 : 3541 : 52
negative
coil_2000 : 586 : 9236 : 85
negative
arrhythmia : 25 : 427 : 278
negative
solar_flare_m0 : 68 : 1321 : 32
negative
oil : 41 : 896 : 49
negative
car_eval_4 : 65 : 1663 : 21
negative
wine_quality : 183 : 4715 : 11
negative
letter_img : 734 : 19266 : 16
negative
yeast_me2 : 51 : 1433 : 8
negative
webpage : 981 : 33799 : 300
negative
ozone_level : 73 : 2463 : 72
negative
mammography : 260 : 10923 : 6
negative
protein_homo : 1296 : 144455 : 74
negative
abalone_19 : 32 : 4145 : 1

In [175]:
for table in table_pos:
    print(np.round(table[115]*100,2))

83.94
92.71
86.59
98.77
81.74
80.21
50.57
99.36
94.03
60.12
52.55
57.73
78.28
69.97
53.69
61.23
58.77
74.76
99.87
47.01
92.48
80.97
67.62
70.91
82.44
73.83
73.23


In [177]:
np.array(table_pos).mean(axis=0)[115]

0.7494046216459381

In [178]:
np.array(table_neg).mean(axis=0)[115]

0.6939013781146705

In [176]:
for table in table_neg:
    print(np.round(table[115]*100,2))

52.07
90.47
58.97
94.54
64.19
61.88
58.32
97.11
76.49
63.35
49.27
54.89
77.61
66.57
53.01
47.95
78.52
68.68
98.79
71.9
88.17
53.1
86.0
48.23
82.32
80.62
50.51
